In [1]:
import os
os.chdir("../")

In [2]:
# Entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

In [3]:
# Configuration Manager
from chestCancerClassifier.constants import *
from chestCancerClassifier.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model

        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )

        return prepare_base_model_config


In [ ]:
# Components
import os
import urllib.request as request
from zipfile import ZipFile
import torch
import torch.nn as nn
import torchvision.models as models

class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config
        if torch.backends.mps.is_available():
            self.device = torch.device("mps")
        elif torch.cuda.is_available():
            self.device = torch.device("cuda")
        else:
            self.device = torch.device("cpu")


    def get_base_model(self):
        # Load VGG16 Model
        self.model = models.vgg16(weights=models.VGG16_Weights[self.config.params_weights])
        if not self.config.params_include_top:
            self.model.classifier = nn.Identity()  # Remove the top layer

        self.model = self.model.to(self.device)
        self.save_model(path=self.config.base_model_path, model=self.model)



    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for param in model.parameters():
                param.requires_grad = False
        elif freeze_all is not None and freeze_till > 0:
            for idx, child in enumerate(model.features.children()):
                if idx < freeze_till:
                    for param in child.parameters():
                        param.requires_grad = False

        full_model = nn.Sequential(
            model,
            nn.Flatten(),
            nn.Linear(25088, classes),
            nn.Softmax(dim=1)
        )

        optimizer = torch.optim.SGD(
            filter(lambda p: p.requires_grad, full_model.parameters()),
            lr=learning_rate
        )
        criterion = nn.CrossEntropyLoss()

        print(full_model)
        return full_model, optimizer, criterion


    def update_base_model(self):
        self.full_model, self.optimizer, self.criterion = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )
        self.full_model.to(self.device)
        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)



    @staticmethod
    def save_model(path: Path, model: nn.Module):
        torch.save(model, path)

In [8]:
# Pipeline
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2025-06-24 01:13:54,150: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-06-24 01:13:54,152: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-24 01:13:54,153: INFO: common: created directory at: artifacts]
[2025-06-24 01:13:54,155: INFO: common: created directory at: artifacts/prepare_base_model]
Sequential(
  (0): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      